# Advanced Topics

This notebook covers advanced features including decomposition, trend/seasonality detection, and autocorrelation analysis.

## What You'll Learn

- Time series decomposition
- Trend detection
- Seasonality detection
- Autocorrelation analysis (ACF/PACF)
- Monte Carlo simulation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from timesmith import (
    autocorrelation,
    partial_autocorrelation,
    monte_carlo_simulation,
)

# Try to import decomposition
try:
    from timesmith.core import (
        DecomposeTransformer,
        detect_trend,
        detect_seasonality,
    )
    HAS_DECOMPOSITION = True
except ImportError:
    HAS_DECOMPOSITION = False
    print("Decomposition not available")

np.random.seed(42)
print("Advanced topics tools loaded!")

## 1. Create Time Series Data

Create a time series with trend, seasonality, and noise.

In [ ]:
# Create time series with trend and seasonality
dates = pd.date_range('2020-01-01', periods=200, freq='D')
trend = np.linspace(100, 150, len(dates))
seasonal = 15 * np.sin(2 * np.pi * np.arange(len(dates)) / 7)
noise = np.random.normal(0, 5, len(dates))
y = pd.Series(trend + seasonal + noise, index=dates)

plt.figure(figsize=(12, 5))
plt.plot(y.index, y.values, linewidth=2)
plt.title('Time Series with Trend and Seasonality', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Data shape: {y.shape}")

## 2. Time Series Decomposition

Decompose time series into trend, seasonal, and residual components.

In [ ]:
if HAS_DECOMPOSITION:
    # Decompose time series
    decomposer = DecomposeTransformer()
    decomposed = decomposer.fit_transform(y)
    
    print(f"Decomposition components: {list(decomposed.columns)}")
    
    # Visualize
    fig, axes = plt.subplots(4, 1, figsize=(12, 10))
    
    axes[0].plot(y.index, y.values, linewidth=2, label='Original')
    axes[0].set_title('Original Series', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    if 'trend' in decomposed.columns:
        axes[1].plot(decomposed.index, decomposed['trend'], linewidth=2, color='orange')
        axes[1].set_title('Trend Component', fontweight='bold')
        axes[1].grid(True, alpha=0.3)
    
    if 'seasonal' in decomposed.columns:
        axes[2].plot(decomposed.index, decomposed['seasonal'], linewidth=2, color='green')
        axes[2].set_title('Seasonal Component', fontweight='bold')
        axes[2].grid(True, alpha=0.3)
    
    if 'residual' in decomposed.columns:
        axes[3].plot(decomposed.index, decomposed['residual'], linewidth=1, color='red', alpha=0.7)
        axes[3].set_title('Residual Component', fontweight='bold')
        axes[3].set_xlabel('Date')
        axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Decomposition not available")

## 3. Trend Detection

Detect the type of trend in the time series.

In [ ]:
if HAS_DECOMPOSITION:
    # Detect trend
    trend_type = detect_trend(y)
    print(f"Detected trend type: {trend_type}")
else:
    print("Trend detection not available")

## 4. Seasonality Detection

Detect if the series has seasonality and its period.

In [ ]:
if HAS_DECOMPOSITION:
    # Detect seasonality
    has_seasonality, period = detect_seasonality(y)
    print(f"Has seasonality: {has_seasonality}")
    if has_seasonality:
        print(f"Seasonal period: {period}")
else:
    print("Seasonality detection not available")

## 5. Autocorrelation Analysis

Compute and visualize ACF and PACF.

In [ ]:
# Compute autocorrelation
acf = autocorrelation(y.values, max_lag=50)
pacf = partial_autocorrelation(y.values, max_lag=50)

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].bar(range(len(acf)), acf, alpha=0.7)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].set_title('Autocorrelation Function (ACF)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')
axes[0].grid(True, alpha=0.3)

axes[1].bar(range(len(pacf)), pacf, alpha=0.7, color='orange')
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('PACF')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"ACF at lag 1: {acf[1]:.4f}")
print(f"ACF at lag 7: {acf[7]:.4f}")

## 6. Monte Carlo Simulation

Use Monte Carlo simulation for uncertainty quantification.

In [ ]:
# Monte Carlo simulation
mc_result = monte_carlo_simulation(y.values, n_simulations=1000, n_steps=30)

print(f"Simulation shape: {mc_result.shape}")
print(f"Mean forecast: {mc_result.mean(axis=0)[-1]:.2f}")
print(f"95% CI: [{np.percentile(mc_result[:, -1], 2.5):.2f}, {np.percentile(mc_result[:, -1], 97.5):.2f}]")

# Visualize sample paths
plt.figure(figsize=(12, 6))
for i in range(min(50, mc_result.shape[0])):
    plt.plot(range(mc_result.shape[1]), mc_result[i, :], 
            alpha=0.1, color='steelblue', linewidth=0.5)

# Plot mean and confidence intervals
mean_path = mc_result.mean(axis=0)
lower = np.percentile(mc_result, 2.5, axis=0)
upper = np.percentile(mc_result, 97.5, axis=0)

plt.plot(range(mc_result.shape[1]), mean_path, 
        linewidth=2, color='red', label='Mean')
plt.fill_between(range(mc_result.shape[1]), lower, upper, 
                alpha=0.3, color='red', label='95% CI')
plt.title('Monte Carlo Simulation Paths', fontsize=14, fontweight='bold')
plt.xlabel('Steps Ahead')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

You've learned:
- How to decompose time series into components
- How to detect trends and seasonality
- How to analyze autocorrelation (ACF/PACF)
- How to use Monte Carlo simulation for uncertainty

**Key Points:**
- Decomposition helps understand time series structure
- ACF/PACF help identify ARIMA model orders
- Monte Carlo simulation provides uncertainty estimates
- These are advanced techniques for deeper analysis